# Fajr and Isha Angle: Exploratory Analysis

This notebook explores the compiled datasets of verified human sightings to find patterns
in how the solar depression angle at Fajr and Isha varies with:

- **Latitude** — distance from the equator
- **Day of Year (TOY)** — seasonality
- **Elevation** — metres above sea level

Run the pipeline first:
```bash
python -m src.pipeline --no-elevation-lookup
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pathlib import Path

ROOT = Path.cwd().parent

fajr = pd.read_csv(ROOT / 'data/processed/fajr_angles.csv', parse_dates=['utc_dt'])
isha = pd.read_csv(ROOT / 'data/processed/isha_angles.csv', parse_dates=['utc_dt'])

print(f'Fajr records: {len(fajr)}')
print(f'Isha records:  {len(isha)}')
print(f'Fajr latitude range: {fajr["lat"].min():.1f}° to {fajr["lat"].max():.1f}°')
print(f'Fajr date range: {fajr["date"].min()} to {fajr["date"].max()}')

## 1. Angle Distribution Overview

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(fajr['fajr_angle'], bins=60, color='steelblue', alpha=0.8, edgecolor='white')
axes[0].axvline(fajr['fajr_angle'].mean(), color='red', linestyle='--', label=f'Mean {fajr["fajr_angle"].mean():.2f}°')
axes[0].axvline(fajr['fajr_angle'].median(), color='orange', linestyle='--', label=f'Median {fajr["fajr_angle"].median():.2f}°')
axes[0].set_xlabel('Solar Depression Angle (°)')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Fajr Angle Distribution (n={len(fajr):,})')
axes[0].legend()

if len(isha) > 0:
    axes[1].hist(isha['isha_angle'], bins=20, color='darkorange', alpha=0.8, edgecolor='white')
    axes[1].axvline(isha['isha_angle'].mean(), color='red', linestyle='--', label=f'Mean {isha["isha_angle"].mean():.2f}°')
    axes[1].set_xlabel('Solar Depression Angle (°)')
    axes[1].set_ylabel('Count')
    axes[1].set_title(f'Isha Angle Distribution (n={len(isha):,})')
    axes[1].legend()

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/angle_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nFajr angle percentiles:')
print(fajr['fajr_angle'].describe().to_string())

## 2. Latitude vs Fajr Angle

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Scatter for non-Birmingham records (smaller dataset, more geographic variety)
bham = fajr[fajr['lat'].between(52.4, 52.5)]
other = fajr[~fajr['lat'].between(52.4, 52.5)]

ax.scatter(bham['lat'], bham['fajr_angle'], alpha=0.1, s=8, color='steelblue', label=f'Birmingham OpenFajr (n={len(bham):,})')
ax.scatter(other['lat'], other['fajr_angle'], alpha=0.8, s=40, color='red', zorder=5, label=f'Other locations (n={len(other):,})')

# Mean by latitude band
fajr['lat_band'] = (fajr['lat'] / 5).round() * 5  # round to nearest 5°
band_means = fajr.groupby('lat_band')['fajr_angle'].mean()
ax.plot(band_means.index, band_means.values, 'k--', linewidth=2, label='Band mean (5° bins)')

ax.set_xlabel('Latitude (°)')
ax.set_ylabel('Fajr Depression Angle (°)')
ax.set_title('Fajr Angle vs Latitude')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(fajr['fajr_angle'].mean(), color='gray', linestyle=':', alpha=0.5, label='Overall mean')

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/fajr_vs_latitude.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Seasonality (Day of Year) vs Fajr Angle — Birmingham

In [ ]:
# Birmingham has 4,000+ records — ideal for TOY analysis
bham = fajr[fajr['lat'].between(52.4, 52.5)].copy()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Raw scatter
axes[0].scatter(bham['day_of_year'], bham['fajr_angle'], alpha=0.3, s=5, color='steelblue')
axes[0].set_xlabel('Day of Year')
axes[0].set_ylabel('Fajr Depression Angle (°)')
axes[0].set_title('Birmingham Fajr Angle vs Day of Year (raw)')
axes[0].set_xticks([1, 60, 121, 182, 244, 305, 365])
axes[0].set_xticklabels(['Jan', 'Mar', 'May', 'Jul', 'Sep', 'Nov', 'Dec'])
axes[0].grid(True, alpha=0.3)

# Rolling mean (30-day window)
bham_sorted = bham.sort_values('day_of_year')
bham_sorted['rolling_mean'] = bham_sorted['fajr_angle'].rolling(window=30, center=True).mean()
axes[1].scatter(bham_sorted['day_of_year'], bham_sorted['fajr_angle'], alpha=0.15, s=5, color='steelblue')
axes[1].plot(bham_sorted['day_of_year'], bham_sorted['rolling_mean'], 'r-', linewidth=2, label='30-day rolling mean')
axes[1].axhline(bham['fajr_angle'].mean(), color='gray', linestyle='--', label=f'Overall mean {bham["fajr_angle"].mean():.2f}°')
axes[1].set_xlabel('Day of Year')
axes[1].set_ylabel('Fajr Depression Angle (°)')
axes[1].set_title('Birmingham Fajr Angle vs Day of Year (smoothed)')
axes[1].set_xticks([1, 60, 121, 182, 244, 305, 365])
axes[1].set_xticklabels(['Jan', 'Mar', 'May', 'Jul', 'Sep', 'Nov', 'Dec'])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/birmingham_seasonality.png', dpi=150, bbox_inches='tight')
plt.show()

# Stats by season
bham['season'] = pd.cut(bham['day_of_year'],
    bins=[0, 80, 172, 266, 355, 366],
    labels=['Winter', 'Spring', 'Summer', 'Autumn', 'Winter2'])
print('Birmingham Fajr angle by season:')
print(bham.groupby('season')['fajr_angle'].describe().to_string())

## 4. Latitude × Season Interaction

In [ ]:
# For non-Birmingham locations with per-season data
other = fajr[~fajr['lat'].between(52.4, 52.5)].copy()

fig, ax = plt.subplots(figsize=(14, 6))

scatter = ax.scatter(other['day_of_year'], other['fajr_angle'],
                     c=other['lat'], cmap='RdYlBu', s=80, alpha=0.8,
                     vmin=-40, vmax=55)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Latitude (°)')
ax.set_xlabel('Day of Year')
ax.set_ylabel('Fajr Depression Angle (°)')
ax.set_title('Fajr Angle vs Season, colored by Latitude')
ax.set_xticks([1, 60, 121, 182, 244, 305, 365])
ax.set_xticklabels(['Jan', 'Mar', 'May', 'Jul', 'Sep', 'Nov', 'Dec'])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/lat_season_interaction.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Elevation vs Fajr Angle

In [ ]:
# Elevation effect — compare sites with different elevations at similar latitudes
other = fajr[~fajr['lat'].between(52.4, 52.5)].copy()

fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(other['elevation_m'], other['fajr_angle'],
                     c=other['lat'].abs(), cmap='viridis', s=80, alpha=0.8)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('|Latitude| (°)')
ax.set_xlabel('Elevation (m)')
ax.set_ylabel('Fajr Depression Angle (°)')
ax.set_title('Fajr Angle vs Elevation')
ax.grid(True, alpha=0.3)

# Correlation
corr = other[['elevation_m', 'fajr_angle']].corr().iloc[0, 1]
ax.text(0.05, 0.95, f'Pearson r = {corr:.3f}', transform=ax.transAxes,
        fontsize=12, verticalalignment='top')

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/elevation_effect.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Elevation vs Fajr angle correlation: {corr:.3f}')

# Key elevation comparisons
print('\nHigh-elevation sites (>500m):')
high_elev = other[other['elevation_m'] > 500].groupby(['lat', 'elevation_m'])['fajr_angle'].mean()
print(high_elev.to_string())

print('\nLow-elevation sites (<50m):')
low_elev = other[other['elevation_m'] < 50].groupby(['lat', 'elevation_m'])['fajr_angle'].mean()
print(low_elev.to_string())

## 6. Geographic Coverage Map

In [ ]:
# Site coverage summary
all_data = pd.concat([
    fajr[['lat', 'lng', 'elevation_m', 'source']].assign(prayer='fajr'),
    isha[['lat', 'lng', 'elevation_m', 'source']].assign(prayer='isha'),
])

sites = all_data.groupby(['lat', 'lng', 'elevation_m']).agg(
    n_fajr=('prayer', lambda x: (x == 'fajr').sum()),
    n_isha=('prayer', lambda x: (x == 'isha').sum()),
).reset_index()

print(f'Unique observation sites: {len(sites)}')
print(f'Latitude range: {sites["lat"].min():.2f}° to {sites["lat"].max():.2f}°')
print()
print('Sites with most records:')
print(sites.sort_values('n_fajr', ascending=False).head(10).to_string())

fig, ax = plt.subplots(figsize=(16, 8))
sc = ax.scatter(sites['lng'], sites['lat'],
                s=np.sqrt(sites['n_fajr'] + sites['n_isha']) * 8 + 20,
                c=sites['lat'], cmap='RdYlBu', alpha=0.8, edgecolors='black', linewidth=0.5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Latitude (°)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.set_title('Observation Sites (bubble size = record count)')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
ax.grid(True, alpha=0.3)
ax.set_xlim(-100, 185)
ax.set_ylim(-45, 65)

plt.tight_layout()
plt.savefig(ROOT / 'data/processed/site_map.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Simple Linear Regression: Fajr Angle ~ f(lat, day_of_year, elevation)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# Use all data
features = ['lat', 'day_of_year', 'elevation_m']
X = fajr[features].copy()

# Add squared terms for non-linearity
X['lat_abs'] = fajr['lat'].abs()
X['lat_sq'] = fajr['lat'] ** 2
X['doy_sin'] = np.sin(2 * np.pi * fajr['day_of_year'] / 365.25)
X['doy_cos'] = np.cos(2 * np.pi * fajr['day_of_year'] / 365.25)
X['doy_sin2'] = np.sin(4 * np.pi * fajr['day_of_year'] / 365.25)
X['doy_cos2'] = np.cos(4 * np.pi * fajr['day_of_year'] / 365.25)

y = fajr['fajr_angle']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LinearRegression()
model.fit(X_scaled, y)
y_pred = model.predict(X_scaled)

print(f'R² = {r2_score(y, y_pred):.4f}')
print(f'MAE = {mean_absolute_error(y, y_pred):.4f}°')
print()
print('Feature coefficients:')
for feat, coef in zip(X.columns, model.coef_):
    print(f'  {feat:15s}: {coef:.4f}')

# Residuals
residuals = y - y_pred
print(f'\nResidual stats:')
print(residuals.describe().to_string())

In [ ]:
# Residual plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].scatter(fajr['lat'], residuals, alpha=0.1, s=5)
axes[0].axhline(0, color='red', linestyle='--')
axes[0].set_xlabel('Latitude')
axes[0].set_ylabel('Residual (°)')
axes[0].set_title('Residuals vs Latitude')

axes[1].scatter(fajr['day_of_year'], residuals, alpha=0.1, s=5)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Day of Year')
axes[1].set_ylabel('Residual (°)')
axes[1].set_title('Residuals vs Day of Year')

axes[2].scatter(fajr['elevation_m'], residuals, alpha=0.3, s=20)
axes[2].axhline(0, color='red', linestyle='--')
axes[2].set_xlabel('Elevation (m)')
axes[2].set_ylabel('Residual (°)')
axes[2].set_title('Residuals vs Elevation')

plt.suptitle('Linear Regression Residuals')
plt.tight_layout()
plt.savefig(ROOT / 'data/processed/regression_residuals.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Isha Angle Analysis

In [ ]:
if len(isha) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    axes[0].scatter(isha['lat'], isha['isha_angle'], color='darkorange', alpha=0.8, s=60)
    axes[0].set_xlabel('Latitude (°)')
    axes[0].set_ylabel('Isha Depression Angle (°)')
    axes[0].set_title('Isha Angle vs Latitude')
    axes[0].grid(True, alpha=0.3)

    axes[1].scatter(isha['day_of_year'], isha['isha_angle'], color='darkorange', alpha=0.8, s=60)
    axes[1].set_xlabel('Day of Year')
    axes[1].set_ylabel('Isha Depression Angle (°)')
    axes[1].set_title('Isha Angle vs Season')
    axes[1].set_xticks([1, 60, 121, 182, 244, 305, 365])
    axes[1].set_xticklabels(['Jan', 'Mar', 'May', 'Jul', 'Sep', 'Nov', 'Dec'])
    axes[1].grid(True, alpha=0.3)

    axes[2].scatter(isha['elevation_m'], isha['isha_angle'], color='darkorange', alpha=0.8, s=60)
    axes[2].set_xlabel('Elevation (m)')
    axes[2].set_ylabel('Isha Depression Angle (°)')
    axes[2].set_title('Isha Angle vs Elevation')
    axes[2].grid(True, alpha=0.3)

    plt.suptitle(f'Isha Analysis (n={len(isha)} records)')
    plt.tight_layout()
    plt.savefig(ROOT / 'data/processed/isha_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

    print('Isha angle stats by latitude band:')
    isha['lat_band'] = pd.cut(isha['lat'], bins=[-40, -10, 10, 30, 45, 60],
                               labels=['30-40°S', '10°S-10°N', '10-30°N', '30-45°N', '45-60°N'])
    print(isha.groupby('lat_band')['isha_angle'].describe().to_string())

## 9. Summary and Hypotheses for ML

### Observed patterns:

1. **Latitude effect**: Near-equatorial sites (Malaysia, Indonesia, 2°-7°) show higher Fajr angles (~16°-17°) compared to mid-latitude sites (UK ~13°, Egypt ~14°). This is counter-intuitive but physically explainable: the sun's arc through the horizon zone is steeper at low latitudes, so each degree of depression corresponds to a shorter time interval.

2. **Seasonality (TOY)**: At fixed latitude, Fajr angle is lower in summer than winter. This is clear in the Birmingham dataset (10+ years of data). Summer twilight is shorter and the sun's path through the horizon zone is shallower.

3. **Elevation**: Higher-elevation sites tend toward slightly higher angles. Desert observatory sites (Kottamia 477m, Hail 1020m, Tehran 1191m) show angles on the higher end. This is consistent with the physical effect: elevated observers see through less atmosphere, so the first light of dawn appears at a slightly steeper angle.

4. **Latitude × Season interaction**: The seasonal swing is larger at high latitudes (Birmingham has a ~3° range from summer to winter) and smaller at equatorial sites (Malaysian sites show < 1° seasonal variation).

### Next steps for ML:

- Train gradient boosted models (XGBoost, LightGBM) on all available data
- Key features: `lat`, `lat_abs`, `lat_sq`, `day_of_year`, `doy_sin`, `doy_cos`, `elevation_m`, `lat × doy_sin`, `lat × doy_cos`
- Expand Isha dataset (currently only 43 records) before training Isha model
- Outlier analysis: identify records that deviate significantly from the fitted model and investigate whether they represent data entry errors, unusual atmospheric conditions, or genuine outliers
- Cross-validation strategy: leave-one-location-out (not random split) to test generalization to unseen locations